# Main Notebook

### Notebook ini akan jadi notebook utama untuk menjalankan semua pengujian-pengujian dengan berbagai parameter

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer #Ini datasset percobaan
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from ffnn.model import FFNN

In [2]:
def load_data():
    # Load breast cancer dataset (binary classification)
    data = load_breast_cancer()
    X = data.data
    y = data.target.reshape(-1, 1)  # Convert to column vector
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Get input dimension
    input_size = X_train.shape[1]
    
    print(f"Dataset: {len(X)} samples, {input_size} features")
    print(f"Training set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")
    
    return X_train, X_test, y_train, y_test, input_size

In [3]:
def train_and_evaluate(X_train, X_test, y_train, y_test, 
                      hidden_layers, width_per_layer, 
                      epochs=50, learning_rate=0.01):

    input_size = X_train.shape[1]
    output_size = 1 
    
    model = FFNN(loss='binary_crossentropy')
    
    # Add input layer to first hidden layer
    if isinstance(width_per_layer, list):
        first_layer_width = width_per_layer[0]
    else:
        first_layer_width = width_per_layer
        
    model.add(input_size=input_size, output_size=first_layer_width, 
             activation='relu', weight_initializer='normal', std=0.1)
    
    for i in range(1, hidden_layers):
        if isinstance(width_per_layer, list):
            prev_width = width_per_layer[i-1]
            current_width = width_per_layer[i]
        else:
            prev_width = width_per_layer
            current_width = width_per_layer
            
        model.add(input_size=prev_width, output_size=current_width,
                 activation='relu', weight_initializer='normal', std=0.1)
    
    # Add output layer with sigmoid activation for binary classification
    if isinstance(width_per_layer, list):
        last_hidden_width = width_per_layer[-1]
    else:
        last_hidden_width = width_per_layer
        
    model.add(input_size=last_hidden_width, output_size=output_size,
             activation='sigmoid', weight_initializer='normal', std=0.1)
    
    # Print model structure
    print(f"\nTraining model with {hidden_layers} hidden layers, width: {width_per_layer}")
    model.summary()
    
    # Train model
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        learning_rate=learning_rate,
        epochs=epochs,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    # Evaluate model
    y_pred = model.forward(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = np.mean(y_pred_binary == y_test)
    
    print(f"Test accuracy: {accuracy:.4f}")
    
    return {
        'model': model,
        'history': history,
        'accuracy': accuracy,
        'depth': hidden_layers,
        'width': width_per_layer
    }